In [1]:
import os
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeout
import time

In [2]:
SEASONS = list(range(2016,2023))

In [3]:
SEASONS 

[2016, 2017, 2018, 2019, 2020, 2021, 2022]

In [4]:
DATA_DIR = "data2"
STANDINGS_DIR = os.path.join(DATA_DIR, "standings")
SCORES_DIR = os.path.join(DATA_DIR, "scores")

In [5]:
async def get_html(url, selector, sleep=5, retries=3):
    html = None
    for i in range(1, retries+1):
        time.sleep(sleep * i)
        
        try:
            async with async_playwright() as p:
                browser = await p.firefox.launch()
                page = await browser.new_page()
                await page.goto(url)
                print(await page.title())
                html = await page.inner_html(selector)
        except PlaywrightTimeout:
            print (f"Timeout error on {url}")
            continue
        else:
            break
    return html

In [6]:
async def scrape_season(season):
    url = f"https://www.basketball-reference.com/leagues/NBA_{season}_games.html"
    html = await get_html(url,"#content .filter")
    
    soup = BeautifulSoup(html)
    links = soup.find_all("a")
    href = [l["href"] for l in links]
    standings_pages = [f"https://basketball-reference.com{l}" for l in href]
    
    for url in standings_pages:
        save_path = os.path.join(STANDINGS_DIR, url.split("/")[-1])
        if os.path.exists(save_path):
            continue
        
        html = await get_html(url, "#all_schedule")
        try: 
            with open(save_path, "w+") as f:
                f.write(html)
        except FileNotFoundError:
            os.makedirs(os.path.dirname(save_path), exist_ok=True)
            with open(save_path, "w+") as f:
                f.write(html)

In [7]:
for season in SEASONS:
    await scrape_season(season)

2015-16 NBA Schedule | Basketball-Reference.com
2016-17 NBA Schedule | Basketball-Reference.com
2017-18 NBA Schedule | Basketball-Reference.com
2018-19 NBA Schedule | Basketball-Reference.com
2019-20 NBA Schedule | Basketball-Reference.com
2020-21 NBA Schedule | Basketball-Reference.com
2021-22 NBA Schedule | Basketball-Reference.com


In [8]:
standings_files = os.listdir(STANDINGS_DIR)

In [9]:
async def scrape_game(standings_file):
    with open(standings_file, 'r') as f:
        html = f.read()

    soup = BeautifulSoup(html)
    links = soup.find_all("a")
    hrefs = [l.get("href") for l in links]
    box_scores = [l for l in hrefs if l and "boxscore" in l and ".html" in l]
    box_scores = [f"https://www.basketball-reference.com{l}" for l in box_scores]

    for url in box_scores:
        save_path = os.path.join(SCORES_DIR, url.split("/")[-1])
        if os.path.exists(save_path):
            continue

        html = await get_html(url, "#content")
        if not html:
            continue
            
        os.makedirs(os.path.dirname(save_path), exist_ok=True)
        with open(save_path, "w+") as f:
                  f.write(html)

In [10]:
SCORES_DIR = 'data2/scores'

In [11]:
standings_files = [s for s in standings_files if ".html" in s]

In [15]:
for f in standings_files:
    filepath = os.path.join(STANDINGS_DIR, f)
    await scrape_game(filepath)

In [13]:
print("finished")

finished
